# Who owns Langstrasse and profits from rising rents?
The Langstrasse was the heart of the red light district in Zurich. In the last 20 years it has changed. Rents have gone up, buildings have been renovated – the whole neighbourhood has been gentrified. But who profits from the rising rents? Have actually companies bought buildings there? The land office of Zurich handed the current owners of the buildings to us – in PDF-format. Let's change that and find a way to analyze the data.

In [1]:
#import the libraries
import pandas as pd
import PyPDF2
import re

In [2]:
#import the AU-PDF
pdfFileObj = open('04-Langstrasse_Eig.Auskunft.pdf', 'rb')
pdfFileObj
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)


#### Read in all the pages of the first PDF
With a loop, read in all the pages and split the text at the kataster-number to get the single buildings.

In [3]:
data = list()
for page in list(range(0,pdfReader.numPages)):
    pageObj = pdfReader.getPage(page)
    langstrasse = pageObj.extractText()
    #print(page)
    langstrasse = langstrasse.replace("\n", "").strip()
    buildings = re.split(r", Kataster", langstrasse) #getting number of listing out
    data.append(buildings)
data[0]

['- 1 - Eigentümerauskunft   Stadtquartier Zürich-Aussersihl 1.   Grundbuch Blatt 1028, Liegenschaft',
 ' AU4987, EGRID CH607891279986, Aussersihl Eigentümer: Herr Fredy Schönholzer, geb. 08.12.1949, Bürgerort: Bürglen TG, Postfach Nr. 47, 6052 Hergiswil NW, Bibimoos, 6052 Hergiswil NW, als Alleineigentü-mer    2.   Grundbuch Blatt 966, Liegenschaft',
 ' AU1093, EGRID CH767199918587, Aussersihl Eigentümer: Herr Bruno Paolo Caretta, geb. 29.06.1947, Bürgerort: Zürich, Hurdnerwäldli-strasse 37, 8808 Pfäffikon SZ, als Miteigentümer zu 1/3  Herr Mauro Claudio Caretta, geb. 26.02.1974, Bürgerort: Zürich, Mülibach-strasse 41, 8805 Richterswil, als Miteigentümer zu 1/3  Herr Rafael Sandro Caretta, geb. 25.06.1975, Bürgerort: Zürich, Militärstrasse 91, 8004 Zürich, als Miteigentümer zu 1/3    3.   Grundbuch Blatt 965, Liegenschaft',
 ' AU1092, EGRID CH549199789993, Aussersihl Eigentümer: Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürgerorte: Splügen GR, Nufenen GR, im Stüdacker 19, 8902 Urdorf, 

## The Loop
- It needs a loop within a loop (for each page and each building)
- Leave out the first one (0) as the first page has additional info on it
- make an empty list to fill up with the data
- split at EGRID and afterwards at ", als"

In [4]:
data_AU = list()

for page in list(range(0,pdfReader.numPages)):
    for building in data[page][1:]:
        places = {
            'kataster': 0,
            'owner': 0
        }
        try: # look at the simple ones
            print(re.split(r", ",building)[0]) #Kataster Nummer
            places['kataster'] = re.split(r", ",building)[0]
            #print(re.split(r"Eigentümer: ", building)[1]) #all owners without XY.Y
            owner = re.split(r"Eigentümer: ", building)[1] #split the owners by Eiegentümer
        except:
            try: #look at the XY.Y ones | where several owner own different stories of one building
                owner = re.split(r"EGRID CH\d* (\w*[\s]*)",building)[2] #split by egrid
            except:
                print("no owner")
        try: #split the owners up, try because there is sometimes no owner
            owner = re.split(r", als", owner) #now split every cell by , als to get all the different owners
            #print(owner)
            #print(len(owner))
        except:
            print("no owner")
        new_list = list()
        if len(owner)!=0: #only save the ownes who have owners
            for i in range(0, len(owner)): #make list of range for different owners
                new_list.append(i-1)
            places['owner']=[owner[own] for own in new_list[1:]] #list comprehension to get all the owners in the dictianory
        #print(places)
        data_AU.append(places) #append to a list

 AU4987
 AU1093
 AU1092
 AU1091
 AU1082
 AU1031
 AU6035
 AU6034
 AU1028
 AU1027
 AU984
 AU983
 AU980
 AU979
 AU978
 AU6741
 AU6740
 AU953
 AU7042
 AU914
 AU5206
 AU903
 AU5444
 AU864
 AU863
 AU845
 AU836
 AU813
 AU812
 AU6403
no owner
no owner
 AU6403
 AU6403
 AU6403
 AU6403
 AU6403
 AU6403
 AU6403
 AU6403
 AU6403
 AU809
 AU804
 AU785
 AU784
 AU783
 AU782
 AU5927
 AU5928
 AU6383
 AU777
 AU5885
 AU766
 AU760
 AU759
no owner
no owner
 AU759
 AU759
 AU759
 AU759
 AU759
 AU759
 AU759
 AU759
 AU759
 AU759
 AU759
 AU871
 AU6948
no owner
no owner
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU6948
 AU5437
 AU5443
 AU6576
 AU7036
 AU7074
 AU6578
 AU4891
 AU4890
 AU1307
 AU1306
 AU1265
 AU5408
 AU1260
 AU4818
 AU1229
 AU1228
 AU1227
 AU1165


Get the addresses of the owners by looking at the end of the string and doing a split.

In [6]:
owner_add = list()
for building in data_AU:
    owner_add = list()
    try: #Stadt Zürich and Erbgemeinschaften don't have addresses
        for i in range(0, len(building['owner'])): #make list of range for different owners
            print(i)
            print(building['kataster'])
            add = re.split(",", building['owner'][i])[-2].strip()+", "+re.split(",", building['owner'][i])[-1].strip()
            owner_add.append(add)
    except:
        if len(building['owner'])>1: #to get the addresses of the single inheritants
            for i in range(1, len(building['owner'])): #make list of range for different owners
                print(i)
                print(building['kataster'])
                add = re.split(",", building['owner'][i])[-2].strip()+", "+re.split(",", building['owner'][i])[-1].strip()
                owner_add.append(add)
        else:  
            print("no address")
            add = "no address"
        owner_add.append(add)
    building.update( {"owner_address": owner_add})


0
 AU4987
0
 AU1093
1
 AU1093
2
 AU1093
0
 AU1092
0
 AU1091
0
 AU1082
1
 AU1082
0
 AU1031
0
 AU6035
0
 AU6034
0
 AU1028
0
 AU1027
0
 AU984
0
 AU983
0
 AU980
0
 AU979
0
 AU978
0
 AU6741
0
 AU6740
0
 AU953
0
 AU7042
1
 AU7042
0
 AU914
0
 AU5206
no address
0
 AU903
0
 AU5444
0
 AU864
0
 AU863
0
 AU845
0
 AU836
0
 AU813
0
 AU812
0
 AU6403
0
 AU6403
0
 AU6403
0
 AU6403
0
 AU6403
0
 AU6403
0
 AU6403
1
 AU6403
0
 AU6403
1
 AU6403
0
 AU6403
0
 AU6403
0
 AU809
0
 AU804
0
 AU785
0
 AU784
1
 AU784
2
 AU784
3
 AU784
0
 AU783
1
 AU783
2
 AU783
0
 AU782
1
 AU782
0
 AU5927
0
 AU5928
0
 AU6383
0
 AU777
0
 AU5885
0
 AU766
0
 AU760
1
 AU760
0
 AU759
1
 AU759
0
 AU759
0
 AU759
0
 AU759
0
 AU759
0
 AU759
1
 AU759
0
 AU759
0
 AU759
0
 AU759
0
 AU759
0
 AU759
1
 AU759
0
 AU759
0
 AU871
0
 AU6948
0
 AU6948
0
 AU6948
0
 AU6948
0
 AU6948
0
 AU6948
0
 AU6948
0
 AU6948
1
 AU6948
0
 AU6948
0
 AU6948
1
 AU6948
0
 AU6948
0
 AU6948
1
 AU6948
0
 AU6948
0
 AU6948
1
 AU6948
0
 AU6948
1
 AU6948
0
 AU6948
0
 AU6948
0
 AU

For the first data frame, the data has been collected. Every Kataster-Number has it's owner. It is still messy. Now put it in a Panda-DF and clean it up afterwards.

In [7]:
df_au = pd.DataFrame()
for owner in data_AU:
    owner_df = pd.DataFrame(owner)
    df_au=pd.concat([df_au, owner_df])
df_au.head()

,kataster,owner,owner_address
0,AU4987,"Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...","Bibimoos, 6052 Hergiswil NW"
0,AU1093,"Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...","Hurdnerwäldli-strasse 37, 8808 Pfäffikon SZ"
1,AU1093,Miteigentümer zu 1/3 Herr Mauro Claudio Care...,"Mülibach-strasse 41, 8805 Richterswil"
2,AU1093,Miteigentümer zu 1/3 Herr Rafael Sandro Care...,"Militärstrasse 91, 8004 Zürich"
0,AU1092,"Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...","im Stüdacker 19, 8902 Urdorf"


Get the "Miteiegntümer, Gesamteigentümer" stuff out

In [8]:
df_au['owner_update1'] = df_au.owner.str.replace(r'.*eigentümer.*  ' , '')

In [9]:
df_au['owner_update1'] = df_au['owner_update1'].str.strip()

Save a new column with only the names of the owner.

In [10]:
df_au['owner_name'] = df_au['owner_update1'].str.split(',',expand=True)[0]

safe a new columne with the additional infos of the owner

In [11]:
df_au['owner_info'] = df_au['owner_update1'].str.split(',', 1,expand=True)[1]
df_au.head()

,kataster,owner,owner_address,owner_update1,owner_name,owner_info
0,AU4987,"Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...","Bibimoos, 6052 Hergiswil NW","Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...",Herr Fredy Schönholzer,"geb. 08.12.1949, Bürgerort: Bürglen TG, Postf..."
0,AU1093,"Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...","Hurdnerwäldli-strasse 37, 8808 Pfäffikon SZ","Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...",Herr Bruno Paolo Caretta,"geb. 29.06.1947, Bürgerort: Zürich, Hurdnerwä..."
1,AU1093,Miteigentümer zu 1/3 Herr Mauro Claudio Care...,"Mülibach-strasse 41, 8805 Richterswil","Herr Mauro Claudio Caretta, geb. 26.02.1974, B...",Herr Mauro Claudio Caretta,"geb. 26.02.1974, Bürgerort: Zürich, Mülibach-..."
2,AU1093,Miteigentümer zu 1/3 Herr Rafael Sandro Care...,"Militärstrasse 91, 8004 Zürich","Herr Rafael Sandro Caretta, geb. 25.06.1975, B...",Herr Rafael Sandro Caretta,"geb. 25.06.1975, Bürgerort: Zürich, Militärst..."
0,AU1092,"Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...","im Stüdacker 19, 8902 Urdorf","Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...",Herr Alfred Hugo Trepp,"geb. 20.11.1943, Bürgerorte: Splügen GR, Nufe..."


## Read in second PDF with IQ

In [12]:
#import the AU-PDF
pdfFileObj = open('05-Langstrasse_Eig.Auskunft.pdf', 'rb')
pdfFileObj
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)


In [13]:
data = list()
for page in list(range(0,pdfReader.numPages)):
    pageObj = pdfReader.getPage(page)
    langstrasse = pageObj.extractText()
    #print(page)
    langstrasse = langstrasse.replace("\n", "").strip()
    buildings = re.split(r", Kataster", langstrasse) #getting number of listing out
    data.append(buildings)

### Repeat the steps from above
First: the loop to get everything out

In [14]:
data_IQ = list()

for page in list(range(0,pdfReader.numPages)):
    for building in data[page][1:]:
        places = {
            'kataster': 0,
            'owner': 0
        }
        try: # look at the simple ones
            print(re.split(r", ",building)[0]) #Kataster Nummer
            places['kataster'] = re.split(r", ",building)[0]
            #print(re.split(r"Eigentümer: ", building)[1]) #all owners without XY.Y
            owner = re.split(r"Eigentümer: ", building)[1] #split the owners by Eiegentümer
        except:
            try: #look at the XY.Y ones
                owner = re.split(r"EGRID CH\d* (\w*[\s]*)",building)[2] #split by egrid
            except:
                print("no owner")
        try: #split the owners up, try because there is sometimes no owner
            owner = re.split(r", als", owner) #now split every cell by , als to get all the different owners
            #print(owner)
            #print(len(owner))
        except:
            print("no owner")
        new_list = list()
        if len(owner)!=0: #only save the ownes who have owners
            for i in range(0, len(owner)): #make list of range for different owners
                new_list.append(i-1)
            places['owner']=[owner[own] for own in new_list[1:]] #list comprehension to get all the owners in the dictianory
        #print(places)
        data_IQ.append(places) #append to a list

 IQ2303
 IQ2245
 IQ2243
 IQ2213
 IQ2166
 IQ2281
 IQ2142
 IQ2138
 IQ2316
 IQ2167
 IQ2282
 IQ2244
 IQ5720
 IQ4000
 IQ2328
 IQ2168
 IQ2280
 IQ4118
 IQ6639
 IQ5719
 IQ6732
 IQ2165
 IQ2178
 IQ2176
 IQ5327
 IQ4398
 IQ2318
 IQ2175
 IQ2177


get the owners address

In [15]:
owner_add = list()
for building in data_IQ:
    owner_add = list()
    try: #Stadt Zürich and Erbgemeinschaften don't have addresses
        for i in range(0, len(building['owner'])): #make list of range for different owners
            print(i)
            print(building['kataster'])
            add = re.split(",", building['owner'][i])[-2].strip()+", "+re.split(",", building['owner'][i])[-1].strip()
            owner_add.append(add)
    except:
        if len(building['owner'])>1: #to get the addresses of the single inheritants
            for i in range(1, len(building['owner'])): #make list of range for different owners
                print(i)
                print(building['kataster'])
                add = re.split(",", building['owner'][i])[-2].strip()+", "+re.split(",", building['owner'][i])[-1].strip()
                owner_add.append(add)
        else:  
            print("no address")
            add = "no address"
        owner_add.append(add)
    building.update( {"owner_address": owner_add})


0
 IQ2303
0
 IQ2245
0
 IQ2243
0
 IQ2213
no address
0
 IQ2166
0
 IQ2281
0
 IQ2142
1
 IQ2142
0
 IQ2138
0
 IQ2316
0
 IQ2167
0
 IQ2282
0
 IQ2244
0
 IQ5720
0
 IQ4000
0
 IQ2328
1
 IQ2328
2
 IQ2328
3
 IQ2328
4
 IQ2328
0
 IQ2168
1
 IQ2168
2
 IQ2168
3
 IQ2168
4
 IQ2168
0
 IQ2280
0
 IQ4118
0
 IQ6639
0
 IQ5719
0
 IQ6732
0
 IQ2165
0
 IQ2178
0
 IQ2176
0
 IQ5327
0
 IQ4398
0
 IQ2318
0
 IQ2175
0
 IQ2177
1
 IQ2177
2
 IQ2177


make a dataframe

In [16]:
df_iq = pd.DataFrame()
for owner in data_IQ:
    owner_df = pd.DataFrame(owner)
    df_iq =pd.concat([df_iq, owner_df])
df_iq.head()

,kataster,owner,owner_address
0,IQ2303,"Herr Konrad Schwarz, geb. 10.12.1972, Bürgeror...","Feldblumenstrasse 19a, 8048 Zürich"
0,IQ2245,"Herr Mario Del Cimmuto, geb. 01.01.1956, Staat...","Wies-lergasse 24, 8049 Zürich"
0,IQ2243,"Herr Heinz Kurt Sönning-John, geb. 10.01.1959,...","Stümelweg 7, 8914 Aeugst am Albis"
0,IQ2213,Stadt Zürich,no address
0,IQ2166,"Frau Dr. iur. Leana Maria Isler, geb. 01.06.19...","Langstrasse 195, 8005 Zürich"


In [17]:
df_iq['owner_update1'] = df_iq.owner.str.replace(r'.*eigentümer.*  ' , '')

In [18]:
df_iq['owner_update1'] = df_iq['owner_update1'].str.strip()

In [19]:
df_iq['owner_name'] = df_iq['owner_update1'].str.split(',',expand=True)[0]

In [20]:
df_iq['owner_info'] = df_iq['owner_update1'].str.split(',', 1,expand=True)[1]

### combine the IQ and the AU dataframes to one

In [21]:
df_pdfs=pd.concat([df_au, df_iq])

In [22]:
df_pdfs.head()

,kataster,owner,owner_address,owner_update1,owner_name,owner_info
0,AU4987,"Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...","Bibimoos, 6052 Hergiswil NW","Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...",Herr Fredy Schönholzer,"geb. 08.12.1949, Bürgerort: Bürglen TG, Postf..."
0,AU1093,"Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...","Hurdnerwäldli-strasse 37, 8808 Pfäffikon SZ","Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...",Herr Bruno Paolo Caretta,"geb. 29.06.1947, Bürgerort: Zürich, Hurdnerwä..."
1,AU1093,Miteigentümer zu 1/3 Herr Mauro Claudio Care...,"Mülibach-strasse 41, 8805 Richterswil","Herr Mauro Claudio Caretta, geb. 26.02.1974, B...",Herr Mauro Claudio Caretta,"geb. 26.02.1974, Bürgerort: Zürich, Mülibach-..."
2,AU1093,Miteigentümer zu 1/3 Herr Rafael Sandro Care...,"Militärstrasse 91, 8004 Zürich","Herr Rafael Sandro Caretta, geb. 25.06.1975, B...",Herr Rafael Sandro Caretta,"geb. 25.06.1975, Bürgerort: Zürich, Militärst..."
0,AU1092,"Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...","im Stüdacker 19, 8902 Urdorf","Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...",Herr Alfred Hugo Trepp,"geb. 20.11.1943, Bürgerorte: Splügen GR, Nufe..."


## Read in new Data to fit Kataster-Nr to an actual address
Source OGD Zurich: http://geoservices.zh.ch/geoshopapi/v1/orders/2d821c17fb3a4213af98ece777124adb/download 

In [24]:
address = pd.read_csv('allzurich.csv')
address.head() #PARZ_NR is equal to the id in the data before
address = address[address['STRASSENNAME']=="Langstrasse"]

In [25]:
address=address.rename(columns = {'PARZ_NR':'kataster'})


In [26]:
df_pdfs['kataster'] = df_pdfs['kataster'].astype(str).str.strip()
address['kataster'] = address['kataster'].astype(str)
address.head()

,BFSNR,ONRP,PLZ,ZUSATZZIFFERN,ORTSCHAFTSNAME,KANTON,STRASSENNAME,HAUSNUMMER,GEOMY,GEOMX,STATUS,STATUS_GP,kataster,GVZ_NR,GWR_EGID,GWR_EDID,GEBAEUDEART,QUELLE,E,N
2792,261,NaN,8004,0,Zürich,NaN,Langstrasse,29,1247762.751,2682036.843,R,NaN,AU871,26401625,148873,1,Gebaeude.Wohngebaeude,AV_2018.05,2682036.843,1247762.751
2793,261,NaN,8004,0,Zürich,NaN,Langstrasse,30,1247754.775,2682059.592,R,NaN,AU5206,26400508,302004639,0,Gebaeude.Verwaltung,AV_2018.05,2682059.592,1247754.775
2938,261,NaN,8004,0,Zürich,NaN,Langstrasse,81,1247997.687,2682145.875,R,NaN,AU1165,26401807,148748,0,Gebaeude.Wohngebaeude,AV_2018.05,2682145.875,1247997.687
2944,261,NaN,8004,0,Zürich,NaN,Langstrasse,113,1248149.039,2682204.337,R,NaN,AU5408,26400972,2366101,0,Gebaeude.Verwaltung,AV_2018.05,2682204.337,1248149.039
3107,261,NaN,8004,0,Zürich,NaN,Langstrasse,118,1248161.862,2682235.118,R,NaN,AU1028,26401623,149327,0,Gebaeude.Wohngebaeude,AV_2018.05,2682235.118,1248161.862


In [27]:
new_df = pd.merge(df_pdfs, address, on='kataster')
new_df


,kataster,owner,owner_address,owner_update1,owner_name,owner_info,BFSNR,ONRP,PLZ,ZUSATZZIFFERN,...,GEOMX,STATUS,STATUS_GP,GVZ_NR,GWR_EGID,GWR_EDID,GEBAEUDEART,QUELLE,E,N
0,AU4987,"Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...","Bibimoos, 6052 Hergiswil NW","Herr Fredy Schönholzer, geb. 08.12.1949, Bürge...",Herr Fredy Schönholzer,"geb. 08.12.1949, Bürgerort: Bürglen TG, Postf...",261,NaN,8004,0,...,2682151.119,R,NaN,26402536,9011685,1,Gebaeude.Wohngebaeude,AV_2018.05,2682151.119,1248010.696
1,AU1093,"Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...","Hurdnerwäldli-strasse 37, 8808 Pfäffikon SZ","Herr Bruno Paolo Caretta, geb. 29.06.1947, Bür...",Herr Bruno Paolo Caretta,"geb. 29.06.1947, Bürgerort: Zürich, Hurdnerwä...",261,NaN,8004,0,...,2682269.710,R,NaN,26400948,149349,0,Gebaeude.Wohngebaeude,AV_2018.05,2682269.710,1248255.859
2,AU1093,Miteigentümer zu 1/3 Herr Mauro Claudio Care...,"Mülibach-strasse 41, 8805 Richterswil","Herr Mauro Claudio Caretta, geb. 26.02.1974, B...",Herr Mauro Claudio Caretta,"geb. 26.02.1974, Bürgerort: Zürich, Mülibach-...",261,NaN,8004,0,...,2682269.710,R,NaN,26400948,149349,0,Gebaeude.Wohngebaeude,AV_2018.05,2682269.710,1248255.859
3,AU1093,Miteigentümer zu 1/3 Herr Rafael Sandro Care...,"Militärstrasse 91, 8004 Zürich","Herr Rafael Sandro Caretta, geb. 25.06.1975, B...",Herr Rafael Sandro Caretta,"geb. 25.06.1975, Bürgerort: Zürich, Militärst...",261,NaN,8004,0,...,2682269.710,R,NaN,26400948,149349,0,Gebaeude.Wohngebaeude,AV_2018.05,2682269.710,1248255.859
4,AU1092,"Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...","im Stüdacker 19, 8902 Urdorf","Herr Alfred Hugo Trepp, geb. 20.11.1943, Bürge...",Herr Alfred Hugo Trepp,"geb. 20.11.1943, Bürgerorte: Splügen GR, Nufe...",261,NaN,8004,0,...,2682266.294,R,NaN,26400968,149331,0,Gebaeude.Verwaltung,AV_2018.05,2682266.294,1248245.247
5,AU1091,"ZH Immobilien AG, mit Sitz in Zürich, CHE-113....","Krönleinstrasse 27, 8044 Zürich","ZH Immobilien AG, mit Sitz in Zürich, CHE-113....",ZH Immobilien AG,"mit Sitz in Zürich, CHE-113.592.562, Krönlein...",261,NaN,8004,0,...,2682262.755,R,NaN,26401802,149330,0,Gebaeude.Wohngebaeude,AV_2018.05,2682262.755,1248234.215
6,AU1082,"Herr lic. phil. Roger Daniel Geissmann, geb. 0...","Langstrasse 132, 8004 Zürich","Herr lic. phil. Roger Daniel Geissmann, geb. 0...",Herr lic. phil. Roger Daniel Geissmann,"geb. 07.05.1960, Bürgerort: Zürich, c/o Resta...",261,NaN,8004,0,...,2682258.415,R,NaN,26401875,149329,0,Gebaeude.Wohngebaeude,AV_2018.05,2682258.415,1248221.895
7,AU1082,"Miteigentümer zu 1/2 Herr Paul Joseph Thoma,...","Oberfeld-ring 13, 8905 Arni-Islisberg","Herr Paul Joseph Thoma, geb. 15.05.1942, Bürge...",Herr Paul Joseph Thoma,"geb. 15.05.1942, Bürgerort: Schänis SG, Oberf...",261,NaN,8004,0,...,2682258.415,R,NaN,26401875,149329,0,Gebaeude.Wohngebaeude,AV_2018.05,2682258.415,1248221.895
8,AU1031,"Herr Urs Peter Humm, geb. 12.10.1942, Bürgeror...","Pilgerweg 5, 8803 Rüschlikon","Herr Urs Peter Humm, geb. 12.10.1942, Bürgeror...",Herr Urs Peter Humm,"geb. 12.10.1942, Bürgerorte: Strengelbach AG,...",261,NaN,8004,0,...,2682248.811,R,NaN,26401000,149328,0,Gebaeude.Wohngebaeude,AV_2018.05,2682248.811,1248196.330
9,AU6035,"Herr Daniel Reitzer, geb. 29.06.1962, Bürgeror...","Lütten-bergstrasse 17, 8906 Bonstetten","Herr Daniel Reitzer, geb. 29.06.1962, Bürgeror...",Herr Daniel Reitzer,"geb. 29.06.1962, Bürgerorte: Rorschach, Basel...",261,NaN,8004,0,...,2682242.959,R,NaN,26401013,2366112,0,Gebaeude.Verwaltung,AV_2018.05,2682242.959,1248184.290


Are the buildings mostly owned by companies or not?

In [28]:
new_df['company'] = new_df['owner_name'].str.contains('|'.join(["Herr", "Frau", "Erben", "Stadt Zürich"])) != True

In [29]:
company = new_df.company.value_counts('normalize')*100
print(round(company[1], 2), "Percent of the buildings on Langstrasse are owned by companies.")

17.59 Percent of the buildings on Langstrasse are owned by companies.


#### are there people who own several buildings? Is there a Misses or a Mister Langstrasse? Or a company that owns a lot?

In [31]:
new_df.owner_name.value_counts().head()

Stadt Zürich                    8
Herr Fredy Schönholzer          4
Herr Urs Peter Humm             3
Herr Morrandas NEVES BOMFIM     3
AM und Partner Immobilien AG    3
Name: owner_name, dtype: int64

-save the data to a csv

In [32]:
new_df.to_csv("langstrasse.csv", index=False)

# Get the Median Income Data
- Using the official API of the Department of Statistics
- Getting the most acutal ones, as they are based on tax-data, there is a four year time lag.

In [33]:
import requests
import time 
import datetime

getting the first 100 Results

In [34]:
url = "https://data.stadt-zuerich.ch/api/3/action/datastore_search?resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac"
response = requests.get(url)
data = response.json()


In [35]:
data['result'].keys()


dict_keys(['include_total', 'resource_id', 'fields', 'records_format', 'records', '_links', 'total'])

In [36]:
income_data = data['result']['records']


In [37]:
df_income = pd.DataFrame(income_data)
df_income.head()

,QuarLang,QuarSort,SteuerEInkommen_p50,SteuerEinkommen_p25,SteuerEinkommen_p75,SteuerJahr,SteuerTarifLang,SteuerTarifSort,_id
0,Rathaus,11,38.9,18.35,64.5,1999,Grundtarif,0,1
1,Rathaus,11,80.4,48.4,127.7,1999,Verheiratetentarif,1,2
2,Rathaus,11,43,23.55,81.1,1999,Einelternfamilientarif,2,3
3,Hochschulen,12,31,12.4,55.2,1999,Grundtarif,0,4
4,Hochschulen,12,86.35,51.7,176.95,1999,Verheiratetentarif,1,5


getting the rest from the api using a while-loop to click next.

In [38]:
url = "https://data.stadt-zuerich.ch"
total_results = data['result']['total']
offset = int(re.findall(r"offset=(\d*)&",data['result']['_links']['next'])[0])


while offset < total_results:
    print(data['result']['_links']['next'])
    offset = int(re.findall(r"offset=(\d*)&",data['result']['_links']['next'])[0])
    response = requests.get(url+data['result']['_links']['next'])
    data = response.json()
    api_results = pd.DataFrame(data['result']['records'])
    df_income = pd.concat([df_income, api_results])


/api/3/action/datastore_search?offset=100&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=200&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=300&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=400&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=500&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=600&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=700&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=800&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=900&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=1000&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496ac
/api/3/action/datastore_search?offset=1100&resource_id=9720e2cc-6a34-452c-8f20-4fc78c9496

now filter to get only single taxed people (most of them).

In [39]:
df = df_income[df_income['SteuerTarifLang'] == "Grundtarif"]

In [40]:
#droping unused columns
df = df.drop(['SteuerEinkommen_p25', 'SteuerEinkommen_p75','SteuerTarifSort','_id'], axis=1)

Writing a function to see the change of the median income between 1999 and 2013.

In [41]:
def get_change(row):
    income_change = (float(row['SteuerEInkommen_p50'])/float(df.SteuerEInkommen_p50[(df['QuarLang']==row['QuarLang']) & (df['SteuerJahr']=='1999')])*100)-100
    return pd.Series({
        'income_change': income_change
        })

In [42]:
df['income_change'] = df.apply(get_change, axis=1)
df = df.sort_values(by=['income_change'], ascending=False)
df_recent = df[df['SteuerJahr']=="2013"]
df_recent.to_csv('income_change.csv', index=False)
df_recent.head(5)

,QuarLang,QuarSort,SteuerEInkommen_p50,SteuerJahr,SteuerTarifLang,income_change
70,Escher Wyss,52,60,2013,Grundtarif,69.014085
91,Seefeld,81,60,2013,Grundtarif,42.857143
58,Werd,41,46.7,2013,Grundtarif,40.662651
61,Langstrasse,42,40.5,2013,Grundtarif,38.461538
49,Alt-Wiedikon,31,50,2013,Grundtarif,36.986301


In the Langstrasse neighbourhood median income has gone up by almost 40 percent during the last 15 years.

##### Now attach this info to the langstrasse df
I have to import the data as I attached the neighbourhood-id via QGIS.

In [43]:
df_langstrasse = pd.read_csv("langstrasse_with_zones.csv")

In [44]:
#attach a building count

In [45]:
def get_buildings_count(row):
    count = len(df_langstrasse[df_langstrasse['owner_name']==row['owner_name']])
    return pd.Series({
        'count': count
        })


In [47]:
df_langstrasse['count'] = df_langstrasse.apply(get_buildings_count, axis=1)

In [48]:
#change colname before merge
df_recent=df_recent.rename(columns = {'QuarLang':'qname_min'})

In [49]:
df = pd.merge(df_langstrasse, df_recent, on='qname_min')


In [50]:
#get list of al colnames to
#erase the cols you are interested in
colnames = list(df.columns.values)
interested_cols = ['kataster','owner_address','owner_name','owner_info','PLZ','GEOMY','STRASSENNAME',
 'HAUSNUMMER','GEOMX','qname_min','kname_min','SteuerEInkommen_p50','SteuerJahr','SteuerTarifLang','income_change','company','count']
[colnames.remove(col) for col in interested_cols]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [51]:
#drop the rest
df_small = df.drop(colnames, axis=1)
df_small.company.replace(False, "No Company", inplace=True)
df_small.company.replace(True, "Company", inplace=True)


In [52]:
df_small.to_csv("df_small.csv", index=False)

1. Open the csv in qgis.
2. Get long lat instead of GEOMX/GEOMY
3. Export from GQIS
4. Load new CSV and go on.

In [57]:
#went to qgis
longlat = pd.read_csv('long_lat_def.csv')
df_small['geometry.coordinates'] = longlat[['X', 'Y']].values.tolist()
df_small['x'] =longlat['X']
df_small['y'] =longlat['Y']
df_small['geometry.type'] = "Point"


## Preparing for Leaflet
Get the information wanted to be displayed in "properties." columns, and the coordinates in the "geometry." columns.

In [58]:
df_final = pd.DataFrame()
def get_all_the_names(row):
    addresse_lang = row['STRASSENNAME']+ " " + row['HAUSNUMMER']
    allenamen = df_small.owner_name[df_small['kataster']==row['kataster']].values
    alleinfos = df_small.owner_address[df_small['kataster']==row['kataster']].values
    name = str()
    owner_all = str()
    for i in range(0,len(allenamen)):
        if name != allenamen[i] + "</br>":
            name = allenamen[i] + "</br>" + name
            owner_all = owner_all + "<b>"+allenamen[i]+"</b>" + "</br>" + alleinfos[i] + "</br>" 

    everything = str("<p>" + owner_all + "</p>" +
  "<p>own(s): " + "<i>" + addresse_lang + " (" + str(row['kataster']) + ")" + "</i>" +
  "<p>" + "Buildings owned in total: " + str(row['count']) + "</p>" +
  "<p>"+"Neighborhood: "+ row['qname_min'] +"</p>" +
  "<p>"+"Change of median income since 1999: "+ str(round(row['income_change'],2))+"%"+"</p>" +
  "<p>"+ "Median income is: " + str(float(row["SteuerEInkommen_p50"])*1000) + " CHF" +"</p>")
    allenamen = name 
    if row['company']=="No Company":
        color_company =  "#3792d6"
    else:      
        color_company = "#efb475"
    return pd.Series({
        'kataster': row['kataster'],
        'xy': str(row['x'] + row['y']),
        'properties.name': addresse_lang,
        'properties.headline': allenamen,
        'properties.article': everything,
        'properties.color': color_company,
        'properties.count': row['count'],
        'geometry.coordinates': row['geometry.coordinates'],
        'geometry.type': row['geometry.type']
        })


Apply the function and save the final data frame as csv.

In [59]:
df_final = df_small.apply(get_all_the_names, axis=1).join(df_final)

In [69]:
df_final = df_final.drop_duplicates('xy', keep='first')
df_final.to_csv('langstrasse_Namen_Adressen.csv')

### Getting the data in geojson (applying Jon's magic)
get the dataframe in propper format for Jon's leaflet template for a Map with points. export geojson file.

In [62]:
import json
#transform your pandas dataframed to json
ok_json = json.loads(df_final.to_json(orient='records'))

In [63]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [274]:
#Pass the file through function
geo_format = process_to_geojson(ok_json)

In [275]:
#This puts in your variable name first
#Then put in the geojson object into your file
with open('geo-data.js', 'w') as outfile:
    outfile.write("infoData = ")
with open('geo-data.js', 'a') as outfile:
    json.dump(geo_format, outfile)